In [87]:
import stan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from scipy.special import expit, logit
import nest_asyncio
nest_asyncio.apply()

In [186]:
model = """
    data {
        int nsites;
        int nsurveys_pc;
        int nsurveys_aru;
        int nsamples; // number of aru recordings
        
        vector[nsites] burn;
        array[nsites, nsurveys_aru] int y_aru;
        array[nsites, nsurveys_pc] int y_pc;
        vector[nsamples] scores;
        array[nsamples] int siteid;
    }
    parameters {
        real<lower=0, upper=1> p11; // PC probability of detection (1) given it is there (1)
        real<lower=0, upper=1> p_aru11; // ARU probability of detection (1) given it is there (1)
        real<lower=0, upper=1> p_aru01; // ARU probability of detection (1) given it is not there (0)
        
        real beta0; // intercept for regression
        real beta1; // slope for regression

        vector[2] mu; // mean for each distribution of scores
        vector[2] sigma; // sd of each distribution of scores
    }
    model {
        vector[nsites] p_pc; // prob of detecting bird with pc
        vector[nsites] p_aru; // prob of detecting bird with aru
        array[nsites] int z;
        vector[nsites] psi;
        
        // priors
        p11 ~ beta(2, 2);
        p_aru11 ~ beta(2, 2);
        p_aru01 ~ beta(1, 3);

        beta0 ~ normal(0, 5);
        beta1 ~ normal(0, 5);

        mu[1] ~ normal(-2, 3);
        mu[2] ~ normal(-2, 3);

        sigma[1] ~ uniform(0.1, 5);
        sigma[2] ~ uniform(0.1, 5);

        psi = beta0 + beta1 * burn;
        z ~ bernoulli_logit(psi);

        // likelihood
        for (i in 1:nsites) {
            p_pc[i] = p11 * z[i];
            for (j in 1:nsurveys_pc) {
                y_pc[i,j] ~ bernoulli(p_pc[i]);
            }

            p_aru[i] = p_aru11 * z[i] + p_aru01;
            for (j in 1:nsurveys_aru) {
                y_aru[i,j] ~ bernoulli(p_aru[i]);
            }
        }

        for (i in 1:nsamples) {
            scores[i] ~ normal(mu[z[siteid[i]] + 1], sigma[z[siteid[i]] + 1]);
        }
    }
"""

In [187]:
# simulate data
def simulation(p11, p_aru11, p_aru01, nvisits, nsites, nrecordings, mu, sigma, beta0, beta1):
    burn = stats.norm.rvs(size=nsites)
    psi_c = expit(beta0 + beta1 * burn)
    z = np.random.binomial(size=nsites, p=psi_c, n=1) # occupancy states

    # generate PC data
    y_ind = np.zeros((nsites, nvisits), dtype=np.uint32)

    for i in range(nsites):
        p_pc = p11 * z[i]
        y_ind[i] = np.random.binomial(size=nvisits, p=p_pc, n=1)
  
    # generate ARU data
    nsamples = nrecordings * nsites

    y_aru = np.zeros((nsites, nrecordings), dtype=np.uint32)

    for i in range(nsites):
        p_aru = z[i] * p_aru11 + p_aru01
        y_aru[i] = np.random.binomial(size=nrecordings, p=p_aru, n=1)


    # simulate scores
    siteids = []
    scores = np.zeros(nsamples)
    for i in range(nsites):
        if nrecordings != 0:
            for j in range(nrecordings):
                index = i * nrecordings + j
                if z[i] == 1:
                    scores[index] = stats.norm.rvs(loc=mu[1], scale=sigma[1], size=1)
                if z[i] == 0:
                    scores[index] = stats.norm.rvs(loc=mu[0], scale=sigma[0], size=1)
                siteids.append(i + 1)

    data = {
        'burn': burn,
        'y_pc': y_ind,
        'y_aru': y_aru, 
        'scores': scores,
        'nsurveys_pc': nvisits,
        'nsurveys_aru': nrecordings,
        'nsites': nsites,
        'nsamples': nsamples,
        'siteid': siteids
    }
    return data

In [188]:
sim = simulation(0.5, 0.5, 0.05, 3, 82, 24, [-2,2], [0.1, 3], 0, 0.5)

In [189]:
posterior = stan.build(model, data=sim, random_seed=1)

Building: 0.2s
Building: 0.3s
Building: 0.4s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.5s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.8s
Building: 6.9s
Building: 7.0s
Building: 

Building: 25.4s, done.
Messages from stanc:
Warning in '/tmp/httpstan_bbvofhxh/model_zbkrfahy.stan', line 62, column 59: The
    variable z may not have been assigned a value before its use.
Warning in '/tmp/httpstan_bbvofhxh/model_zbkrfahy.stan', line 62, column 34: The
    variable z may not have been assigned a value before its use.
Warning in '/tmp/httpstan_bbvofhxh/model_zbkrfahy.stan', line 55, column 33: The
    variable z may not have been assigned a value before its use.
Warning in '/tmp/httpstan_bbvofhxh/model_zbkrfahy.stan', line 50, column 28: The
    variable z may not have been assigned a value before its use.
Warning in '/tmp/httpstan_bbvofhxh/model_zbkrfahy.stan', line 46, column 8: The
    variable z may not have been assigned a value before its use.


In [190]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%


RuntimeError: Exception during call to services function: `RuntimeError("Initialization between (-2, 2) failed after 100 attempts. Try specifying initial values, reducing ranges of constrained values, or reparameterizing the model. Initialization failed. Rejecting initial value: Error evaluating the log probability at the initial value. Exception: bernoulli_logit_lpmf: n[1] is -2147483648, but must be in the interval [0, 1] (in '/tmp/httpstan_ehoxldss/model_zbkrfahy.stan', line 46, column 8 to column 33) Rejecting initial value:")`, traceback: `['  File "/home/mschulist/miniconda3/lib/python3.10/asyncio/tasks.py", line 232, in __step\n    result = coro.send(None)\n', '  File "/home/mschulist/miniconda3/lib/python3.10/site-packages/httpstan/services_stub.py", line 182, in call\n    raise RuntimeError(exception_message)\n']`